<a href="https://colab.research.google.com/github/yssscz/Shicheng-Yan-DS-project/blob/main/ds_5220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# load data
train_data = pd.read_csv("train.tsv", sep="\t")
test_data = pd.read_csv("test.tsv", sep="\t")

# check
print(train_data.head())


# get the text and label
train_texts = train_data["Phrase"].tolist()
train_labels = train_data["Sentiment"].tolist()

# split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42
)

# test dataset
test_texts = test_data["Phrase"].tolist()


   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  


In [4]:

print(train_data['Sentiment'].value_counts())


Sentiment
2    79582
3    32927
1    27273
4     9206
0     7072
Name: count, dtype: int64


In [7]:
import re

def remove_special_characters(text):
    # remove special characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text

In [35]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
  # remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def clean_text_pipeline(text):
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    return text

In [11]:
train_data["Cleaned_Phrase"] = train_data["Phrase"].apply(clean_text_pipeline)

print(train_data.head())

   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment                                     Cleaned_Phrase  
0          1  A series escapades demonstrating adage good go...  
1          2  A series escapades demonstrating adage good goose  
2          2                                           A series  
3          2                                                  A  
4          2                                             series  


In [14]:
def remove_duplicates_and_empty(data): # clean dataset

    data = data[data['Cleaned_Phrase'].str.strip() != ""]

    data = data.drop_duplicates(subset=['Cleaned_Phrase'])

    return data

In [21]:
def clean_single_word_and_sentiment(data):
# Delete all rows where the Cleaned_Phrase column contains only one word and the Sentiment value is 2.
    data['Cleaned_Phrase'] = data['Cleaned_Phrase'].fillna("")

    data = data[~((data['Cleaned_Phrase'].str.split().str.len() <= 2) & (data['Sentiment'] == 2))]

    return data
cleaned_train_data2 = clean_single_word_and_sentiment(cleaned_train_data)

In [22]:
print(cleaned_train_data2['Sentiment'].value_counts())

Sentiment
2    19436
3    18901
1    15953
4     5331
0     4139
Name: count, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
# cleaned dataset split
X_train, X_val, y_train, y_val = train_test_split(
    cleaned_train_data2['Cleaned_Phrase'],
    cleaned_train_data2['Sentiment'],
    test_size=0.2,
    random_state=42
)


In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, accuracy_score
import torch

# Load the BERT tokenizer
bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Data preprocessing function
def preprocess_function(texts, labels=None):
    encodings = tokenizer(list(texts), padding="max_length", truncation=True, max_length=256)
    if labels is not None:
        encodings["labels"] = torch.tensor(list(labels))
    return encodings

# Using the cleaned dataset
train_encodings = preprocess_function(X_train, y_train)
val_encodings = preprocess_function(X_val, y_val)

# Load the BERT model
model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=5)

# Define Trainer arguments
training_args = TrainingArguments(
    output_dir="./bert_results",
    overwrite_output_dir=True,
    run_name="bert_finetune_experiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Reduce batch size to save GPU memory
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./bert_logs",
)

# Define custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Validation set predictions
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(-1)  # Get predicted classes

# Compute evaluation metrics
print("Accuracy:", accuracy_score(y_val, preds))
print(
    classification_report(
        y_val,
        preds,
        target_names=["Negative", "Somewhat Negative", "Neutral", "Somewhat Positive", "Positive"]
    )
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-28-26f8022ff3c1>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
<ipython-input-28-26f8022ff3c1>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.927400,0.929026
2,0.760800,0.889293
3,0.634900,0.942031


<ipython-input-28-26f8022ff3c1>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-26f8022ff3c1>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-26f8022ff3c1>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-28-26f8022ff3c1>:48: UserWarning: To copy construct fro

Accuracy: 0.6378607277289837
                   precision    recall  f1-score   support

         Negative       0.53      0.44      0.48       813
Somewhat Negative       0.63      0.67      0.65      3207
          Neutral       0.66      0.64      0.65      3877
Somewhat Positive       0.66      0.69      0.67      3768
         Positive       0.58      0.51      0.54      1087

         accuracy                           0.64     12752
        macro avg       0.61      0.59      0.60     12752
     weighted avg       0.64      0.64      0.64     12752



In [31]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# Load the GPT2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=5)

# set Padding Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Data preprocessing function
def preprocess_function(texts, labels=None, max_len=128):
    encodings = tokenizer(
        list(texts),
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    if labels is not None:
        encodings["labels"] = torch.tensor(list(labels), dtype=torch.long)
    return encodings

# using the cleaned dataset
train_encodings = preprocess_function(X_train, y_train, max_len=128)
val_encodings = preprocess_function(X_val, y_val, max_len=128)

# Define custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer arguments
training_args = TrainingArguments(
    output_dir="./gpt2_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./gpt2_logs",
    logging_steps=10,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate()
print("Evaluation results:", results)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-31-247096d681b8>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.072400,0.981343,0.588457,0.589023,0.588457,0.581645
2,0.825600,0.936919,0.609395,0.606210,0.609395,0.603755
3,0.681300,0.944515,0.607748,0.605373,0.607748,0.604371


Evaluation results: {'eval_loss': 0.9369191527366638, 'eval_accuracy': 0.6093946047678795, 'eval_precision': 0.6062095339221956, 'eval_recall': 0.6093946047678795, 'eval_f1': 0.603755266881116, 'eval_runtime': 22.903, 'eval_samples_per_second': 556.783, 'eval_steps_per_second': 69.598, 'epoch': 3.0}


In [32]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Predict on the validation set using the trained model
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=-1)  # Get predicted classes

# Print the shape of the predictions (to check alignment)
print("Predictions shape:", preds.shape)

# Print the shape of the validation labels
print("Validation labels shape:", np.array(y_val).shape)

# Ensure predictions and true labels are aligned
if len(preds) < len(y_val):
    y_val = y_val[:len(preds)]
elif len(preds) > len(y_val):
    preds = preds[:len(y_val)]

# Output classification report
report = classification_report(
    y_val, preds, target_names=["Negative", "Somewhat Negative", "Neutral", "Somewhat Positive", "Positive"]
)
print("Classification Report:\n", report)

# Output overall accuracy
accuracy = accuracy_score(y_val, preds)
print("Overall Accuracy:", accuracy)


Predictions shape: (12752,)
Validation labels shape: (12752,)
Classification Report:
                    precision    recall  f1-score   support

         Negative       0.51      0.34      0.41       813
Somewhat Negative       0.59      0.61      0.60      3207
          Neutral       0.63      0.65      0.64      3877
Somewhat Positive       0.61      0.69      0.65      3768
         Positive       0.58      0.36      0.45      1087

         accuracy                           0.61     12752
        macro avg       0.59      0.53      0.55     12752
     weighted avg       0.61      0.61      0.60     12752

Overall Accuracy: 0.6093946047678795


In [3]:
# using the uncleaned dataset
# Load BERT model
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=5)

# Define Trainer arguments
training_args = TrainingArguments(
    output_dir="./bert_results",
    run_name="bert_finetune_experiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./bert_logs",
)

# Define dataset format
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

train_dataset1 = CustomDataset(train_encodings)
val_dataset1 = CustomDataset(val_encodings)

# Initialize Trainer
bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset1,
    eval_dataset=val_dataset1,
)

# Fine-tune the model
bert_trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yscz (yscz-northeastern-university). Use `wandb login --relogin` to force relogin


<ipython-input-3-73e8d4eab3d0>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.880200,0.721229
2,0.682200,0.706213
3,0.624500,0.712110


<ipython-input-3-73e8d4eab3d0>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-3-73e8d4eab3d0>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-3-73e8d4eab3d0>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=2928, training_loss=0.7078721093349769, metrics={'train_runtime': 674.5333, 'train_samples_per_second': 555.264, 'train_steps_per_second': 4.341, 'total_flos': 4.927466077551821e+16, 'train_loss': 0.7078721093349769, 'epoch': 3.0})

In [6]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Predict on the validation set
bert_predictions = bert_trainer.predict(val_dataset1)
bert_preds = bert_predictions.predictions.argmax(-1)  # Get predicted classes

# Extract validation labels from encoded data
val_labels = val_encodings["labels"].tolist()  # Ensure labels align with the validation set

# Check the size of validation labels and predictions
print("Number of validation samples:", len(val_labels))
print("Number of predictions:", len(bert_preds))

# Align predictions and true labels
if len(bert_preds) < len(val_labels):
    val_labels = val_labels[:len(bert_preds)]
elif len(bert_preds) > len(val_labels):
    bert_preds = bert_preds[:len(val_labels)]

# Print evaluation results
print("Accuracy:", accuracy_score(val_labels, bert_preds))  # Output accuracy
print(classification_report(val_labels, bert_preds, target_names=["Negative", "Somewhat Negative", "Neutral", "Somewhat Positive", "Positive"]))


<ipython-input-3-73e8d4eab3d0>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


验证集样本数: 31212
预测结果数: 31212
Accuracy: 0.7036716647443291
                   precision    recall  f1-score   support

         Negative       0.55      0.50      0.52      1416
Somewhat Negative       0.60      0.63      0.62      5527
          Neutral       0.80      0.80      0.80     15639
Somewhat Positive       0.64      0.62      0.63      6707
         Positive       0.57      0.62      0.59      1923

         accuracy                           0.70     31212
        macro avg       0.63      0.63      0.63     31212
     weighted avg       0.70      0.70      0.70     31212



In [34]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=5)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# using the uncleaned dataset
X_train, X_val, y_train, y_val = train_test_split(
    train_data["Phrase"],
    train_data["Sentiment"],
    test_size=0.2,
    random_state=42
)

def preprocess_function(texts, labels=None, max_len=128):
    encodings = tokenizer(
        list(texts),
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    if labels is not None:
        encodings["labels"] = torch.tensor(list(labels), dtype=torch.long)
    return encodings


train_encodings = preprocess_function(X_train, y_train, max_len=128)
val_encodings = preprocess_function(X_val, y_val, max_len=128)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./gpt2_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./gpt2_logs",
    logging_steps=10,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate()
print("Evaluation results:", results)

predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=-1)
report = classification_report(
    y_val, preds.numpy(), target_names=["Negative", "Somewhat Negative", "Neutral", "Somewhat Positive", "Positive"]
)
print("Classification Report:\n", report)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-34-66dedcc91213>:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.919200,0.764746,0.688806,0.681548,0.688806,0.674943
2,0.783300,0.742043,0.696847,0.692453,0.696847,0.692914
3,0.668900,0.763674,0.696367,0.694510,0.696367,0.695097


Evaluation results: {'eval_loss': 0.7420432567596436, 'eval_accuracy': 0.6968473663975394, 'eval_precision': 0.6924528072449029, 'eval_recall': 0.6968473663975394, 'eval_f1': 0.6929139269443868, 'eval_runtime': 55.4307, 'eval_samples_per_second': 563.082, 'eval_steps_per_second': 70.394, 'epoch': 3.0}
Classification Report:
                    precision    recall  f1-score   support

         Negative       0.61      0.37      0.46      1416
Somewhat Negative       0.60      0.61      0.60      5527
          Neutral       0.77      0.82      0.79     15639
Somewhat Positive       0.63      0.60      0.61      6707
         Positive       0.58      0.57      0.57      1923

         accuracy                           0.70     31212
        macro avg       0.64      0.59      0.61     31212
     weighted avg       0.69      0.70      0.69     31212



In [24]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.metrics import accuracy_score, classification_report

# 数据集类定义
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels.iloc[index]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# 加载 BERT 分词器和模型
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

# 参数设置
batch_size = 64
max_len = 128
epochs = 3
learning_rate = 2e-5

# 数据加载器
train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_len=max_len)
val_dataset = SentimentDataset(X_val, y_val, tokenizer, max_len=max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# 优化器和调度器
optimizer = AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# 设备选择
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练和验证循环
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # ======= 训练阶段 =======
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        # 获取输入和标签
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # 前向传播
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss:.4f}")

    # ======= 验证阶段 =======
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            # 前向传播
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_val_loss += loss.item()

            # 获取预测结果
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

# ======= 结果报告 =======
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Somewhat Negative", "Neutral", "Somewhat Positive", "Positive"]))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Training Loss: 1.0517
Validation Loss: 0.9253
Validation Accuracy: 0.6074
Epoch 2/3
Training Loss: 0.8378
Validation Loss: 0.9059
Validation Accuracy: 0.6190
Epoch 3/3
Training Loss: 0.7374
Validation Loss: 0.9047
Validation Accuracy: 0.6302

Classification Report:
                   precision    recall  f1-score   support

         Negative       0.51      0.42      0.46       813
Somewhat Negative       0.61      0.66      0.64      3207
          Neutral       0.66      0.63      0.65      3877
Somewhat Positive       0.65      0.68      0.67      3768
         Positive       0.56      0.52      0.54      1087

         accuracy                           0.63     12752
        macro avg       0.60      0.58      0.59     12752
     weighted avg       0.63      0.63      0.63     12752

